In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 101 kB 8.7 MB/s 
     |████████████████████████████████| 596 kB 50.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer
import torch 

In [4]:
from ast import literal_eval

## 导入数据

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/情感打分数据包/Chinese_after_ig_score_前7000条.csv")
df

,txt,year,text_length,txt_seg,Topic,Doc_ID,text,nopunc
0,本报记者季芳16日，广州亚运会的射击场上，出现了一个久违的身影。38岁的杨凌在阔别赛场多年之...,2010.11.17,684,本报记者 季芳 16 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 38...,24,45807,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...
1,沈秀赵渊在异彩纷呈的中国彩电业中，创维电视是一匹令人瞩目的“黑马”。在国内市场，创维仅用了5...,1999.03.01,682,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,251,45813,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...
2,◆中国地区大学野球（１６日、山口・周南市野球場）東亜大３―０吉備国際大徳山大２―０福山大,2007.04.17,44,中国 地区 大学 野球 日 山口 周 南市 野球 場 東亜 大 吉備 国際大 徳 山大 福山 大,4,45818,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大
3,本报讯综合本报和新华社驻外记者报道：我国神舟六号载人航天飞行圆满成功在国际社会引起强烈反响。...,2005.10.18,690,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,32,45819,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...
4,美国决定不增加日棉天鹅绒进口税新华社24日讯美国总统艾森豪威尔在22日拒绝了美国关税委员会要...,1957.01.25,416,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 24 日讯 美国 总统 艾森豪威尔 在 ...,215,45824,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...
...,...,...,...,...,...,...,...,...
495,本报讯国际乒乓球联合会２２日在哥德堡举行了第四十二届世界乒乓球锦标赛男女团体比赛的分组抽签。...,1993.04.23,234,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,24,49011,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...
496,美帝侵略亚洲的战争阴谋必将失败越外交部发言人强烈谴责美国空军轰炸老挝桑怒解放区新华社河内二十...,1965.03.01,699,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,87,49023,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...
497,在大沽口外的海洋上，有着一批无畏的人们同风浪搏斗着，争取更多地超额完成国家计划。他们是天津区...,1958.01.01,461,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,142,49031,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...
498,中国增七枚日本添六枚南朝鲜加四枚本报汉城23日专电今天亚运会的射击、跳水、体操、游泳、举重和...,1986.09.24,562,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,24,49033,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...


## 定义模型

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [ ]:
# 加载 BertForSequenceClassification, 预训练 BERT 模型 + 顶层的线性分类层 
model = BertForSequenceClassification.from_pretrained(
#     "model/bert-base-chinese-FineTuned-Binary-Best", # 小写的 12 层预训练模型
    "/content/drive/MyDrive/轨道/情感--bert--fine-tuning/result_6", # 小写的 12 层预训练模型
    num_labels = 2, # 分类数 --2 表示二分类
                    # 你可以改变这个数字，用于多分类任务  
#     output_attentions = False, # 模型是否返回 attentions weights.
#     output_hidden_states = False, # 模型是否返回所有隐层状态.
)
model.to(device)
model.eval()
model.zero_grad()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/轨道/情感--bert--fine-tuning/result_6')

### 定义分段打分（以512字节为界）函数

In [ ]:
import torch.nn.functional as F
def split_chunk(text): #为什么要split_chunk, 512的数值有意义吗
    str = text
#     str = re.sub(r"\s+", "", str)
    str = str.strip()
    n = 512
    chunks = []
    i = 0
    while i < len(str):
        if i+n < len(str):
            chunks.append(str[i:i+n])
        else:
            chunks.append(str[i:len(str)])
        i += n
    return(chunks)

#直接得到每一段的积极消极分类
def getlb(txt):
    chunk_sample=split_chunk(txt)
    res = []
    for i in range(len(chunk_sample)):
        inp = tokenizer(chunk_sample[i], return_tensors='pt', padding="max_length", max_length=512)
        output = model(**inp)
        # output
        # output = F.softmax(output.pooler_output[0])
        embeding = output.logits.detach().numpy()
        lb = np.argmax(embeding)
        res.append(lb)
    return res

#获得每一段消极的分数
def get_neg_score(txt):
    chunk_sample=split_chunk(txt)
    res = []
    for i in range(len(chunk_sample)):
        inp = tokenizer(chunk_sample[i], return_tensors='pt', padding="max_length", max_length=512)
        output = model(**inp)
        # output
        # output = F.softmax(output.pooler_output[0])
        embeding = F.softmax(output.logits).detach().numpy()
        neg_score = embeding[0][0]
        res.append(neg_score)
    return res

#获得每一段积极的分数
def get_pst_score(txt):
    chunk_sample=split_chunk(txt)
    res = []
    for i in range(len(chunk_sample)):
        inp = tokenizer(chunk_sample[i], return_tensors='pt', padding="max_length", max_length=512)
        output = model(**inp)
        # output
        # output = F.softmax(output.pooler_output[0])
        embeding = F.softmax(output.logits).detach().numpy()
        pst_score = embeding[0][1]
        res.append(pst_score)
    return res

#获得每一段积极的分数
def get_all(txt):
    chunk_sample=split_chunk(txt)
    res = []
    for i in range(len(chunk_sample)):
        inp = tokenizer(chunk_sample[i], return_tensors='pt', padding="max_length", max_length=512)
        output = model(**inp)
        # output
        # output = F.softmax(output.pooler_output[0])
        embeding = F.softmax(output.logits).detach().numpy()
        pst_score = embeding[0][1]#积极分数
        neg_score = embeding[0][0]#消极分数
        lb = np.argmax(embeding)#情感分类
        res.append(pst_score, neg_score, lb)
    return res

### 根据需求求得新闻的情感（分类、积极消极具体的分数）

In [ ]:
#分类
df['label'] = df.nopunc.apply(getlb)
df

,txt,year,text_length,txt_seg,Topic,Doc_ID,text,nopunc,label
0,本报记者季芳16日，广州亚运会的射击场上，出现了一个久违的身影。38岁的杨凌在阔别赛场多年之...,2010.11.17,684,本报记者 季芳 16 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 38...,24,45807,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,"[1, 1]"
1,沈秀赵渊在异彩纷呈的中国彩电业中，创维电视是一匹令人瞩目的“黑马”。在国内市场，创维仅用了5...,1999.03.01,682,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,251,45813,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,"[1, 1]"
2,◆中国地区大学野球（１６日、山口・周南市野球場）東亜大３―０吉備国際大徳山大２―０福山大,2007.04.17,44,中国 地区 大学 野球 日 山口 周 南市 野球 場 東亜 大 吉備 国際大 徳 山大 福山 大,4,45818,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,[1]
3,本报讯综合本报和新华社驻外记者报道：我国神舟六号载人航天飞行圆满成功在国际社会引起强烈反响。...,2005.10.18,690,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,32,45819,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,"[1, 1]"
4,美国决定不增加日棉天鹅绒进口税新华社24日讯美国总统艾森豪威尔在22日拒绝了美国关税委员会要...,1957.01.25,416,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 24 日讯 美国 总统 艾森豪威尔 在 ...,215,45824,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,"[1, 0]"
...,...,...,...,...,...,...,...,...,...
495,本报讯国际乒乓球联合会２２日在哥德堡举行了第四十二届世界乒乓球锦标赛男女团体比赛的分组抽签。...,1993.04.23,234,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,24,49011,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,[0]
496,美帝侵略亚洲的战争阴谋必将失败越外交部发言人强烈谴责美国空军轰炸老挝桑怒解放区新华社河内二十...,1965.03.01,699,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,87,49023,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,"[0, 0]"
497,在大沽口外的海洋上，有着一批无畏的人们同风浪搏斗着，争取更多地超额完成国家计划。他们是天津区...,1958.01.01,461,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,142,49031,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,"[1, 1]"
498,中国增七枚日本添六枚南朝鲜加四枚本报汉城23日专电今天亚运会的射击、跳水、体操、游泳、举重和...,1986.09.24,562,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,24,49033,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,"[1, 1]"


In [ ]:
#消极分数
df['negative_score'] = df.nopunc.apply(get_neg_score)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


,txt,year,text_length,txt_seg,Topic,Doc_ID,text,nopunc,label,negative_score
0,本报记者季芳16日，广州亚运会的射击场上，出现了一个久违的身影。38岁的杨凌在阔别赛场多年之...,2010.11.17,684,本报记者 季芳 16 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 38...,24,45807,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,"[1, 1]","[0.07522548, 0.089829646]"
1,沈秀赵渊在异彩纷呈的中国彩电业中，创维电视是一匹令人瞩目的“黑马”。在国内市场，创维仅用了5...,1999.03.01,682,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,251,45813,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,"[1, 1]","[0.053187795, 0.05040593]"
2,◆中国地区大学野球（１６日、山口・周南市野球場）東亜大３―０吉備国際大徳山大２―０福山大,2007.04.17,44,中国 地区 大学 野球 日 山口 周 南市 野球 場 東亜 大 吉備 国際大 徳 山大 福山 大,4,45818,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,[1],[0.1449167]
3,本报讯综合本报和新华社驻外记者报道：我国神舟六号载人航天飞行圆满成功在国际社会引起强烈反响。...,2005.10.18,690,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,32,45819,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,"[1, 1]","[0.09707586, 0.090314195]"
4,美国决定不增加日棉天鹅绒进口税新华社24日讯美国总统艾森豪威尔在22日拒绝了美国关税委员会要...,1957.01.25,416,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 24 日讯 美国 总统 艾森豪威尔 在 ...,215,45824,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,"[1, 0]","[0.16100405, 0.8853508]"
...,...,...,...,...,...,...,...,...,...,...
495,本报讯国际乒乓球联合会２２日在哥德堡举行了第四十二届世界乒乓球锦标赛男女团体比赛的分组抽签。...,1993.04.23,234,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,24,49011,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,[0],[0.5972218]
496,美帝侵略亚洲的战争阴谋必将失败越外交部发言人强烈谴责美国空军轰炸老挝桑怒解放区新华社河内二十...,1965.03.01,699,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,87,49023,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,"[0, 0]","[0.9497963, 0.7810614]"
497,在大沽口外的海洋上，有着一批无畏的人们同风浪搏斗着，争取更多地超额完成国家计划。他们是天津区...,1958.01.01,461,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,142,49031,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,"[1, 1]","[0.046765257, 0.05514983]"
498,中国增七枚日本添六枚南朝鲜加四枚本报汉城23日专电今天亚运会的射击、跳水、体操、游泳、举重和...,1986.09.24,562,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,24,49033,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,"[1, 1]","[0.05419384, 0.09429364]"


In [ ]:
#积极分数
df['positicve_score'] = df.nopunc.apply(get_pst_score)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


,txt,year,text_length,txt_seg,Topic,Doc_ID,text,nopunc,label,negative_score,positicve_score
0,本报记者季芳16日，广州亚运会的射击场上，出现了一个久违的身影。38岁的杨凌在阔别赛场多年之...,2010.11.17,684,本报记者 季芳 16 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 38...,24,45807,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,"[1, 1]","[0.07522548, 0.089829646]","[0.92477447, 0.9101704]"
1,沈秀赵渊在异彩纷呈的中国彩电业中，创维电视是一匹令人瞩目的“黑马”。在国内市场，创维仅用了5...,1999.03.01,682,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,251,45813,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,"[1, 1]","[0.053187795, 0.05040593]","[0.94681215, 0.9495941]"
2,◆中国地区大学野球（１６日、山口・周南市野球場）東亜大３―０吉備国際大徳山大２―０福山大,2007.04.17,44,中国 地区 大学 野球 日 山口 周 南市 野球 場 東亜 大 吉備 国際大 徳 山大 福山 大,4,45818,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,[1],[0.1449167],[0.8550833]
3,本报讯综合本报和新华社驻外记者报道：我国神舟六号载人航天飞行圆满成功在国际社会引起强烈反响。...,2005.10.18,690,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,32,45819,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,"[1, 1]","[0.09707586, 0.090314195]","[0.9029242, 0.90968585]"
4,美国决定不增加日棉天鹅绒进口税新华社24日讯美国总统艾森豪威尔在22日拒绝了美国关税委员会要...,1957.01.25,416,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 24 日讯 美国 总统 艾森豪威尔 在 ...,215,45824,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,"[1, 0]","[0.16100405, 0.8853508]","[0.83899593, 0.11464912]"
...,...,...,...,...,...,...,...,...,...,...,...
495,本报讯国际乒乓球联合会２２日在哥德堡举行了第四十二届世界乒乓球锦标赛男女团体比赛的分组抽签。...,1993.04.23,234,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,24,49011,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,[0],[0.5972218],[0.4027782]
496,美帝侵略亚洲的战争阴谋必将失败越外交部发言人强烈谴责美国空军轰炸老挝桑怒解放区新华社河内二十...,1965.03.01,699,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,87,49023,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,"[0, 0]","[0.9497963, 0.7810614]","[0.0502037, 0.21893859]"
497,在大沽口外的海洋上，有着一批无畏的人们同风浪搏斗着，争取更多地超额完成国家计划。他们是天津区...,1958.01.01,461,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,142,49031,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,"[1, 1]","[0.046765257, 0.05514983]","[0.9532348, 0.94485015]"
498,中国增七枚日本添六枚南朝鲜加四枚本报汉城23日专电今天亚运会的射击、跳水、体操、游泳、举重和...,1986.09.24,562,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,24,49033,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,"[1, 1]","[0.05419384, 0.09429364]","[0.94580615, 0.90570635]"


In [ ]:
df

,txt,year,text_length,txt_seg,Topic,Doc_ID,text,nopunc,label,negative_score,positicve_score
0,本报记者季芳16日，广州亚运会的射击场上，出现了一个久违的身影。38岁的杨凌在阔别赛场多年之...,2010.11.17,684,本报记者 季芳 16 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 38...,24,45807,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,"[1, 1]","[0.07522548, 0.089829646]","[0.92477447, 0.9101704]"
1,沈秀赵渊在异彩纷呈的中国彩电业中，创维电视是一匹令人瞩目的“黑马”。在国内市场，创维仅用了5...,1999.03.01,682,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,251,45813,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,"[1, 1]","[0.053187795, 0.05040593]","[0.94681215, 0.9495941]"
2,◆中国地区大学野球（１６日、山口・周南市野球場）東亜大３―０吉備国際大徳山大２―０福山大,2007.04.17,44,中国 地区 大学 野球 日 山口 周 南市 野球 場 東亜 大 吉備 国際大 徳 山大 福山 大,4,45818,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,[1],[0.1449167],[0.8550833]
3,本报讯综合本报和新华社驻外记者报道：我国神舟六号载人航天飞行圆满成功在国际社会引起强烈反响。...,2005.10.18,690,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,32,45819,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,"[1, 1]","[0.09707586, 0.090314195]","[0.9029242, 0.90968585]"
4,美国决定不增加日棉天鹅绒进口税新华社24日讯美国总统艾森豪威尔在22日拒绝了美国关税委员会要...,1957.01.25,416,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 24 日讯 美国 总统 艾森豪威尔 在 ...,215,45824,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,"[1, 0]","[0.16100405, 0.8853508]","[0.83899593, 0.11464912]"
...,...,...,...,...,...,...,...,...,...,...,...
495,本报讯国际乒乓球联合会２２日在哥德堡举行了第四十二届世界乒乓球锦标赛男女团体比赛的分组抽签。...,1993.04.23,234,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,24,49011,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,[0],[0.5972218],[0.4027782]
496,美帝侵略亚洲的战争阴谋必将失败越外交部发言人强烈谴责美国空军轰炸老挝桑怒解放区新华社河内二十...,1965.03.01,699,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,87,49023,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,"[0, 0]","[0.9497963, 0.7810614]","[0.0502037, 0.21893859]"
497,在大沽口外的海洋上，有着一批无畏的人们同风浪搏斗着，争取更多地超额完成国家计划。他们是天津区...,1958.01.01,461,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,142,49031,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,"[1, 1]","[0.046765257, 0.05514983]","[0.9532348, 0.94485015]"
498,中国增七枚日本添六枚南朝鲜加四枚本报汉城23日专电今天亚运会的射击、跳水、体操、游泳、举重和...,1986.09.24,562,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,24,49033,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,"[1, 1]","[0.05419384, 0.09429364]","[0.94580615, 0.90570635]"


In [ ]:
df.to_excel("/content/drive/MyDrive/情感打分数据包/6500到11000条情感标签(无9500).xlsx",index=False)
df

### 直接得到上面三个指标

In [ ]:
#df['label'] = 0
#df['negative_score'] = 0
#df['positicve_score'] = 0
#for i in range(len(df)):
    #df.label[i], df.positicve_score[i], df.negative_score[i] = get_all(df.iloc[i, 1])
#df

### 如果要这篇新闻的情感，可以对上面的指标求平均

In [ ]:
def getmean(list_t):
    lt = np.array(list_t)
    return np.mean(lt)

In [ ]:
#积极分数--平均
df['positicve_score__mean'] = df.positicve_score.apply(getmean)
df

,txt,year,text_length,txt_seg,Topic,Doc_ID,text,nopunc,label,negative_score,positicve_score,positicve_score__mean
0,本报记者季芳16日，广州亚运会的射击场上，出现了一个久违的身影。38岁的杨凌在阔别赛场多年之...,2010.11.17,684,本报记者 季芳 16 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 38...,24,45807,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,"[1, 1]","[0.07522548, 0.089829646]","[0.92477447, 0.9101704]",0.917472
1,沈秀赵渊在异彩纷呈的中国彩电业中，创维电视是一匹令人瞩目的“黑马”。在国内市场，创维仅用了5...,1999.03.01,682,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,251,45813,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,"[1, 1]","[0.053187795, 0.05040593]","[0.94681215, 0.9495941]",0.948203
2,◆中国地区大学野球（１６日、山口・周南市野球場）東亜大３―０吉備国際大徳山大２―０福山大,2007.04.17,44,中国 地区 大学 野球 日 山口 周 南市 野球 場 東亜 大 吉備 国際大 徳 山大 福山 大,4,45818,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,[1],[0.1449167],[0.8550833],0.855083
3,本报讯综合本报和新华社驻外记者报道：我国神舟六号载人航天飞行圆满成功在国际社会引起强烈反响。...,2005.10.18,690,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,32,45819,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,"[1, 1]","[0.09707586, 0.090314195]","[0.9029242, 0.90968585]",0.906305
4,美国决定不增加日棉天鹅绒进口税新华社24日讯美国总统艾森豪威尔在22日拒绝了美国关税委员会要...,1957.01.25,416,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 24 日讯 美国 总统 艾森豪威尔 在 ...,215,45824,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,"[1, 0]","[0.16100405, 0.8853508]","[0.83899593, 0.11464912]",0.476823
...,...,...,...,...,...,...,...,...,...,...,...,...
495,本报讯国际乒乓球联合会２２日在哥德堡举行了第四十二届世界乒乓球锦标赛男女团体比赛的分组抽签。...,1993.04.23,234,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,24,49011,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,[0],[0.5972218],[0.4027782],0.402778
496,美帝侵略亚洲的战争阴谋必将失败越外交部发言人强烈谴责美国空军轰炸老挝桑怒解放区新华社河内二十...,1965.03.01,699,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,87,49023,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,"[0, 0]","[0.9497963, 0.7810614]","[0.0502037, 0.21893859]",0.134571
497,在大沽口外的海洋上，有着一批无畏的人们同风浪搏斗着，争取更多地超额完成国家计划。他们是天津区...,1958.01.01,461,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,142,49031,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,"[1, 1]","[0.046765257, 0.05514983]","[0.9532348, 0.94485015]",0.949042
498,中国增七枚日本添六枚南朝鲜加四枚本报汉城23日专电今天亚运会的射击、跳水、体操、游泳、举重和...,1986.09.24,562,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,24,49033,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,"[1, 1]","[0.05419384, 0.09429364]","[0.94580615, 0.90570635]",0.925756


In [ ]:
#消极分数--平均
df['negative_score_mean'] = df.negative_score.apply(getmean)
df

,txt,year,text_length,txt_seg,Topic,Doc_ID,text,nopunc,label,negative_score,positicve_score,positicve_score__mean,negative_score_mean
0,本报记者季芳16日，广州亚运会的射击场上，出现了一个久违的身影。38岁的杨凌在阔别赛场多年之...,2010.11.17,684,本报记者 季芳 16 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 38...,24,45807,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,本报记者 季芳 日 广州 亚运会 的 射击场 上 出现 了 一个 久违 的 身影 岁 的 杨...,"[1, 1]","[0.07522548, 0.089829646]","[0.92477447, 0.9101704]",0.917472,0.082528
1,沈秀赵渊在异彩纷呈的中国彩电业中，创维电视是一匹令人瞩目的“黑马”。在国内市场，创维仅用了5...,1999.03.01,682,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,251,45813,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,沈秀 赵渊 在 异彩纷呈 的 中国 彩电业 中 创维 电视 是 一匹 令人瞩目 的 黑马 在...,"[1, 1]","[0.053187795, 0.05040593]","[0.94681215, 0.9495941]",0.948203,0.051797
2,◆中国地区大学野球（１６日、山口・周南市野球場）東亜大３―０吉備国際大徳山大２―０福山大,2007.04.17,44,中国 地区 大学 野球 日 山口 周 南市 野球 場 東亜 大 吉備 国際大 徳 山大 福山 大,4,45818,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,中国 地区 大学 野球 日 山口 周 南市 野球 場東 亜 大吉 備国際 大 徳 山大 福山 大,[1],[0.1449167],[0.8550833],0.855083,0.144917
3,本报讯综合本报和新华社驻外记者报道：我国神舟六号载人航天飞行圆满成功在国际社会引起强烈反响。...,2005.10.18,690,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,32,45819,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,本报讯 综合 本报 和 新华社 驻外 记者 报道 我国 神舟 六号 载人 航天 飞行 圆满成...,"[1, 1]","[0.09707586, 0.090314195]","[0.9029242, 0.90968585]",0.906305,0.093695
4,美国决定不增加日棉天鹅绒进口税新华社24日讯美国总统艾森豪威尔在22日拒绝了美国关税委员会要...,1957.01.25,416,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 24 日讯 美国 总统 艾森豪威尔 在 ...,215,45824,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,美国 决定 不 增加 日棉 天鹅绒 进口税 新华社 日讯 美国 总统 艾森豪威尔 在 日 拒...,"[1, 0]","[0.16100405, 0.8853508]","[0.83899593, 0.11464912]",0.476823,0.523177
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,本报讯国际乒乓球联合会２２日在哥德堡举行了第四十二届世界乒乓球锦标赛男女团体比赛的分组抽签。...,1993.04.23,234,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,24,49011,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,本报讯 国际 乒乓球 联合会 日 在 哥德堡 举行 了 第四十二届 世界 乒乓球 锦标赛 男...,[0],[0.5972218],[0.4027782],0.402778,0.597222
496,美帝侵略亚洲的战争阴谋必将失败越外交部发言人强烈谴责美国空军轰炸老挝桑怒解放区新华社河内二十...,1965.03.01,699,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,87,49023,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,美帝 侵略 亚洲 的 战争 阴谋 必将 失败 越 外交部 发言人 强烈 谴责 美国空军 轰炸...,"[0, 0]","[0.9497963, 0.7810614]","[0.0502037, 0.21893859]",0.134571,0.865429
497,在大沽口外的海洋上，有着一批无畏的人们同风浪搏斗着，争取更多地超额完成国家计划。他们是天津区...,1958.01.01,461,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,142,49031,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,在 大沽口 外 的 海洋 上 有着 一批 无畏 的 人们 同 风浪 搏斗 着 争取 更多地 ...,"[1, 1]","[0.046765257, 0.05514983]","[0.9532348, 0.94485015]",0.949042,0.050958
498,中国增七枚日本添六枚南朝鲜加四枚本报汉城23日专电今天亚运会的射击、跳水、体操、游泳、举重和...,1986.09.24,562,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,24,49033,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,中国 增 七枚 日本 添 六枚 南朝鲜 加 四枚 本报 汉城 日 专电 今天 亚运会 的 射...,"[1, 1]","[0.05419384, 0.09429364]","[0.94580615, 0.90570635]",0.925756,0.074244


## 保持新的文件

In [ ]:
df.to_csv("/content/drive/MyDrive/情感打分数据包/含标签/情感标签_7000.csv",index=False)
#df.to_excel("/content/drive/MyDrive/情感打分数据包/chinese_6500到11000_score_（无9500）含情感标签.xlsx",index=False)